# Localization_Classification.ipynb
Date: November 29th, 2018  
Course: ECSE415, McGill University  
Authors:  
*Shawn Vosburg  
Tristan Bouchard  
Alex Masciotra  
Nayem Alam  
Thomas Philippon *

## 3. Localization & Classification

In [7]:
#Import the required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import  random as rand
import csv


In [59]:
#Class hoding the dimensions of the object windows.  
class rectangle():
    def __init__(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        
#Image directory 
folder = "./MIO-TCD-Localization/Localization/train/"
csv_datapath = "./MIO-TCD-Localization/MIO-TCD-Localization/gt_train.csv"

#load the .csv file 
#with open('./MIO-TCD-Localization/MIO-TCD-Localization/gt_train.csv') as f: 
    #reader = csv.reader(f)
   # for i in reader:
    #    print(i)

#Stucture holding the labels 
vehicleName = [
    "articulated_truck", "bicycle",
    "bus", "car", "motorcycle","motorized_vehicle " "non-motorized_vehicle",
    "pedestrian", "pickup_truck", "single_unit_truck", "work_van"                
]


#function reading a .csv file and creating a rectangle object from the GT table
def read_image(img_id, csv_datapath):
    
    """Function that takes an image from the dataset and the GT csv file and returns an array of rectangles each detected object on the image 
    ----------
    img_id : image name (name of the file) 
    csv_file : csv file read using the csv.read function
    Returns 
    -------
    rectangle_list : list of rectangle objects for each detected object on the image 
    """
    #create the list of rectangles object 
    rectangle_list = []
    
    #open the csv file and retrieve the gt
    with open(csv_datapath) as f: 
        reader = csv.reader(f)
        for i in reader:
            if(i[0] == img_id): 
                rect = rectangle(x = i[2], y = i[4] , h = i[3] ,w = i[5])
                rectangle_list.append(rect)
    
    return rectangle_list 


#function to compute the dice coefficients 
def dice_coef(gt_list, loc_list):
    
    """Function that takes the detected windows from the localization algorithm and computes the dice coefficients using the gt table
    ----------
    csv_file : csv file read using the csv.read function
   
    Returns 
    -------
    dice_list : array of rectangle objects for each detected object on the image 
    """
    dice_list = [] 
    
    return dice_list 


a = read_image("00000000", csv_datapath)


194
